In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import c, e, m_p

%matplotlib tk
import seaborn as sns
sns.set_context('notebook', font_scale=1.5)
sns.set_style('darkgrid', {
        'axes.linewidth': 2,
        'lines.markeredgewidth': 0,
        'legend.fancybox': True})

/usr/lib64/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


## Bunch creation

We create a bunch - LHC type. No fancy wrapper function available yet to produce a RF matched distribution. A little tedious because it requires quite some definitions to be done beforehand of the full RF System - but that's due to the nature of the problem.

- Some beam kinetic parameters and RF machine parameters

In [2]:
from PyHEADTAIL.trackers.rf_bucket import RFBucket
from PyHEADTAIL.particles.generators import ParticleGenerator, \
    gaussian2D, RF_bucket_distribution, StationaryExponential

PyHEADTAIL v1.7.2.1




In [3]:
momentum = 450e9 * e/c
gamma = np.sqrt((momentum/(m_p*c))**2 + 1)
beta = np.sqrt(1 - gamma**-2)

circumference = 26658.883
alpha = 3.225e-4
h1 = 35640
V1 = 8e6
p_increment = 0 * e/c * circumference/(beta*c)

print gamma

479.606062093


In [4]:
rfbucket = RFBucket(
    charge=e, mass=m_p, gamma=gamma,
    circumference=circumference,
    alpha_array=[alpha], p_increment=0,
    harmonic_list=[h1], voltage_list=[V1], phi_offset_list=[0])

In [5]:
bunch = ParticleGenerator(
    macroparticlenumber=1e5, intensity=1e11,
    charge=e, mass=m_p, gamma=gamma,
    circumference=26658.883,
    distribution_x=gaussian2D(2e-6),
    distribution_y=gaussian2D(2e-6),
    distribution_z=RF_bucket_distribution(rfbucket, epsn_z=0.35)
).generate()

*** Maximum RMS emittance 0.888689919173eV s.
... distance to target emittance: 2.32e-02
... distance to target emittance: 2.34e-02
... distance to target emittance: -5.11e-04
... distance to target emittance: 1.04e-05
--> Emittance: 0.350000004218
--> Bunch length:0.0697492609648


In [6]:
bunch2 = ParticleGenerator(
    macroparticlenumber=1e5, intensity=1e11,
    charge=e, mass=m_p, gamma=gamma,
    circumference=26658.883,
    distribution_x=gaussian2D(2e-6),
    distribution_y=gaussian2D(2e-6),
    distribution_z=RF_bucket_distribution(rfbucket, epsn_z=0.35)
).generate()

distance = 3e-9 * (bunch2.beta*c)
bunch2.z -= distance

*** Maximum RMS emittance 0.888689919173eV s.
... distance to target emittance: 2.32e-02
... distance to target emittance: 2.34e-02
... distance to target emittance: -5.11e-04
... distance to target emittance: 1.04e-05
--> Emittance: 0.350000004218
--> Bunch length:0.0697492609648


In [7]:
# fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(6,6))
# ax1.plot(bunch.x, bunch.xp, '.')
# ax2.plot(bunch.y, bunch.yp, '.')
# ax3.plot(bunch.z, bunch.dp, '.')
# ax3.set_ylim(-max(abs(bunch.dp))-1e-4, max(abs(bunch.dp))+1e-4)
# plt.show()

In [8]:
bunches = bunch + bunch2

plt.plot(bunches.z, bunches.dp, '.')
plt.show()

## Wakes, slicing, convolution etc.

The CircularResonator wake is a special resonator wake with Yokoya factors X1=1, Y1=1, X2=0, Y2=0

functions transverse and longitudinal are actually meant to be used internally when building the wake kicks - at this point they are concatenated with the respective Yokoya factor being applied/provided. For visualisation we can also build them externally manually.

In [9]:
from PyHEADTAIL.impedances.wakes import CircularResonator, \
    WakeField, check_wake_sampling
from PyHEADTAIL.particles.slicing import UniformBinSlicer

In [10]:
wake = CircularResonator(R_shunt=1e9, frequency=1e9, Q=20)
slicer = UniformBinSlicer(50, z_cuts=(-.3, .3))

wakefields = WakeField(slicer, wake)
kick = wakefields.wake_kicks[0]
wf = kick.wake_function

slices = bunch.get_slices(slicer)
times = slices.z_centers / (bunch.beta*c)
dt = distance/(bunches.beta*c)

bunch2.z += distance
slices2 = bunch2.get_slices(slicer)
times2 = slices2.z_centers / (bunch2.beta*c)
bunch2.z -= distance

*** PyHEADTAIL WARNING! Acceleration not handled properly by this kind of convolution due to changing bunch length!
*** PyHEADTAIL WARNING! Acceleration not handled properly by this kind of convolution due to changing bunch length!


In [11]:
fig, (ax1, ax2) = plt.subplots(2, figsize=(12,6))

[ax1.axvline(s, c='r') for s in slices.z_centers]
[ax1.axvline(s, c='b') for s in [slices.z_cut_tail, slices.z_cut_head]]
ax1.plot(slices.z_centers, slices.charge_per_slice, '-o')

[ax2.axvline(s, c='r') for s in slices2.z_centers]
[ax2.axvline(s, c='b') for s in [slices2.z_cut_tail, slices2.z_cut_head]]
ax2.plot(slices2.z_centers, slices2.charge_per_slice, '-o')

# ax1.set_xlim(-1, 1)
plt.show()

## Convolutions

### First bunch

In [12]:
delta_xp = 0.*times
for i, s in enumerate(slices.charge_per_slice):
    for j in range(slices.n_slices)[::-1]:
        delta_xp[i] += slices.charge_per_slice[j]*wf(times[i]-times[j])

In [13]:
# tt = times - times[0]
# tt = np.concatenate((-tt[::-1], tt[1:]))
t0 = np.concatenate((times-times[-1], (times-times[0])[1:]))

dxp = np.convolve(slices.charge_per_slice, wf(t0))
dxp_s = np.convolve(slices.charge_per_slice, wf(t0), mode='same')
dxp_v = np.convolve(slices.charge_per_slice, wf(t0), mode='valid')

fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(12,10))

ax1.plot(t0, wf(t0)[::-1]/max(wf(t0)[::-1]))
ax1.plot(times-times[-1], slices.charge_per_slice/max(slices.charge_per_slice))
ax1.set_ylim((-1.1, 1.1))
ax1.legend(['Bunch', 'Wake'])
ax1.set_xlim(-8e-9, 3e-9)
ax2.plot(delta_xp)
ax2.plot(dxp_v)
ax2.legend(['manual', 'valid'])
ax3.plot(times-times[-1], slices.charge_per_slice/max(slices.charge_per_slice))
ax3.plot(times-times[-1], dxp_v/max(dxp_v))
ax3.set_ylim(-2, 2)

plt.show()

dxp_v1 = dxp_v.copy()

### Second bunch

In [14]:
delta_xp = 0.*times
for i, s in enumerate(slices.charge_per_slice):
    for j in range(slices.n_slices)[::-1]:
        delta_xp[i] += slices2.charge_per_slice[j]*wf(times[i]-times[j])
        delta_xp[i] += slices.charge_per_slice[j]*wf(times[i]-times[j] - dt)

In [15]:
t0 = np.concatenate((times-times[-1], (times-times[0])[1:]))
t2 = t0 - dt

dxp = np.convolve(slices2.charge_per_slice, wf(t0))
dxp_s = np.convolve(slices2.charge_per_slice, wf(t0), mode='same')
dxp_v = np.convolve(slices2.charge_per_slice, wf(t0), mode='valid')

dxp += np.convolve(slices.charge_per_slice, wf(t2))
dxp_s += np.convolve(slices.charge_per_slice, wf(t2), mode='same')
dxp_v += np.convolve(slices.charge_per_slice, wf(t2), mode='valid')

fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(12,10))

ax1.plot(t0, wf(t0)[::-1]/max(wf(t0)[::-1]))
ax1.plot(times-times[-1]-dt, slices2.charge_per_slice/max(slices2.charge_per_slice))
ax1.set_ylim((-1.1, 1.1))
ax1.legend(['Bunch', 'Wake'])
ax1.set_xlim(-8e-9, 3e-9)
ax2.plot(delta_xp)
ax2.plot(dxp_v)
ax2.legend(['manual', 'valid'])
ax3.plot(times-times[-1], slices2.charge_per_slice/max(slices2.charge_per_slice))
ax3.plot(times-times[-1], dxp_v/max(dxp_v))
ax3.set_ylim(-2, 2)

plt.show()

dxp_v2 = dxp_v.copy()

### More bunches

In [16]:
allslicer = UniformBinSlicer(300, z_cuts=(-2, .3))
allslices = bunches.get_slices(allslicer)

alltimes = allslices.z_centers / (bunches.beta*c)

In [17]:
delta_xp = 0.*alltimes
for i, s in enumerate(allslices.charge_per_slice):
    for j in range(allslices.n_slices)[::-1]:
        delta_xp[i] += allslices.charge_per_slice[j]*wf(alltimes[i]-alltimes[j])

In [18]:
t0 = np.concatenate((alltimes-alltimes[-1],
                    (alltimes-alltimes[0])[1:]))

dxp = np.convolve(allslices.charge_per_slice, wf(t0))
dxp_s = np.convolve(allslices.charge_per_slice, wf(t0), mode='same')
dxp_v = np.convolve(allslices.charge_per_slice, wf(t0), mode='valid')

fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(12,10))

ax1.plot(t0, wf(t0)[::-1]/max(wf(t0)[::-1]))
ax1.plot(alltimes-alltimes[-1],
         allslices.charge_per_slice/max(allslices.charge_per_slice))
ax1.set_ylim((-1.1, 1.1))
ax1.legend(['Bunch', 'Wake'])
ax1.set_xlim(-8e-9, 3e-9)
ax2.plot(delta_xp)
ax2.plot(dxp_v)
ax2.legend(['manual', 'valid'])

ax3.plot(alltimes-alltimes[-1],
         allslices.charge_per_slice/max(allslices.charge_per_slice))
ax3.plot(alltimes-alltimes[-1], dxp_v/max(dxp_v), label="all bunches")

ax3.plot(times-times[-1]-dt, slices2.charge_per_slice/max(slices2.charge_per_slice))
ax3.plot(times-times[-1]-dt, dxp_v2/max(dxp_v), label="second bunch")

ax3.plot(times-times[-1], slices.charge_per_slice/max(slices.charge_per_slice))
ax3.plot(times-times[-1], dxp_v1/max(dxp_v), label="first bunch")

ax3.legend()
ax3.set_ylim(-2, 2)

# c = sns.color_palette('husl', 3)
# [ax3.axvline(s, c=c[0], alpha=0.8) for s in (times-times[-1])]
# [ax3.axvline(s, c=c[1], alpha=0.8) for s in (times2-times2[-1]-dt)]
# [ax3.axvline(s, c=c[2], alpha=0.8) for s in (alltimes-alltimes[-1])]

plt.show()

In [ ]:
plt.close('all')